# Tile mosaic image acquisitions from Opera Phenix 

This notebook exhibits the original tiling code used to compile Opera Phenix acquisition into contiguous mosaic images for tracking cells across tiles. 

### Original approach

In [1]:
from macrohet import dataio, tile

/home/dayn/analysis/macrohet/macrohet/tile.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
metadata = dataio.read_harmony_metadata('../data/untiled_images/Index_0305.idx.xml')

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


In [3]:
metadata

,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0305K1F1P1R1,Ok,r03c05f01p01-ch1sk1fk1fl1.tiff,3,5,1,1,0,1,1,...,0,0.135621503,0,2021-04-16T19:11:18.61+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0305K1F1P1R2,Ok,r03c05f01p01-ch2sk1fk1fl1.tiff,3,5,1,1,0,2,1,...,0,0.135621503,0,2021-04-16T19:11:18.61+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0305K1F1P2R1,Ok,r03c05f01p02-ch1sk1fk1fl1.tiff,3,5,1,2,0,1,1,...,2E-06,0.1356235,0,2021-04-16T19:11:18.89+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0305K1F1P2R2,Ok,r03c05f01p02-ch2sk1fk1fl1.tiff,3,5,1,2,0,2,1,...,2E-06,0.1356235,0,2021-04-16T19:11:18.89+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0305K1F1P3R1,Ok,r03c05f01p03-ch1sk1fk1fl1.tiff,3,5,1,3,0,1,1,...,4E-06,0.135625601,0,2021-04-16T19:11:19.17+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,0305K75F9P1R2,Ok,r03c05f09p01-ch2sk75fk1fl1.tiff,3,5,9,1,74,2,1,...,0,0.135624304,266400.56,2021-04-19T21:11:30.447+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4045,0305K75F9P2R1,Ok,r03c05f09p02-ch1sk75fk1fl1.tiff,3,5,9,2,74,1,1,...,2E-06,0.135626301,266400.56,2021-04-19T21:11:30.713+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4046,0305K75F9P2R2,Ok,r03c05f09p02-ch2sk75fk1fl1.tiff,3,5,9,2,74,2,1,...,2E-06,0.135626301,266400.56,2021-04-19T21:11:30.713+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4047,0305K75F9P3R1,Ok,r03c05f09p03-ch1sk75fk1fl1.tiff,3,5,9,3,74,1,1,...,4E-06,0.135628298,266400.56,2021-04-19T21:11:30.993+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


In [4]:
metadata.keys()

Index(['id', 'State', 'URL', 'Row', 'Col', 'FieldID', 'PlaneID', 'TimepointID',
       'ChannelID', 'FlimID', 'ChannelName', 'ImageType', 'AcquisitionType',
       'IlluminationType', 'ChannelType', 'ImageResolutionX',
       'ImageResolutionY', 'ImageSizeX', 'ImageSizeY', 'BinningX', 'BinningY',
       'MaxIntensity', 'CameraType', 'PositionX', 'PositionY', 'PositionZ',
       'AbsPositionZ', 'MeasurementTimeOffset', 'AbsTime',
       'MainExcitationWavelength', 'MainEmissionWavelength',
       'ObjectiveMagnification', 'ObjectiveNA', 'ExposureTime',
       'OrientationMatrix'],
      dtype='object')

In [5]:
metadata['ImageSizeX'].iloc[0]

'2160'

In [21]:
float(metadata['ImageSizeX'].iloc[0])/5.04

428.57142857142856

In [89]:
metadata['ImageResolutionX'].iloc[0]

'1.4949402023919043E-07'

In [17]:
from skimage import io

io.imread('../data/untiled_images/r03c05f01p01-ch1sk66fk1fl1.tiff').shape

(428, 428)

In [3]:
import numpy as np
import pandas as pd

columns_to_scale = [
    'ImageSizeX', 'ImageSizeY',
    # 'ImageResolutionX', 'ImageResolutionY',
    'PositionX', 'PositionY', 'PositionZ'
]

# Ensure they're numeric, floor-divide, cast to int then str
metadata[columns_to_scale] = (
    metadata[columns_to_scale]
    .apply(pd.to_numeric, errors='coerce')
    .div(5.04)
    .apply(np.floor)
    .astype(int)
    .astype(str)
)


In [4]:
image_dir = '../data/untiled_images/'
images = tile.compile_mosaic(image_dir, metadata, row=3, col=5,  n_tile_cols=3, n_tile_rows=3)

Stacked shape: (450, 428, 1198)
Expected reshape: (75, 2, 3, 428, 1198)


In [5]:
images

dask.array<reshape, shape=(75, 2, 3, 1198, 1198), dtype=uint16, chunksize=(1, 1, 1, 428, 428), chunktype=numpy.ndarray>

In [6]:
images[0,0,0].compute()

ValueError: total size of new array must be unchanged

In [8]:
images.shape

(75, 2, 3, 428, 1198)

In [8]:
images.compute_chunk_sizes()

ValueError: total size of new array must be unchanged

In [9]:
images[0].shape

(2, 3, 428, 1198)

In [17]:
%%time
test = images[0].compute().compute()

CPU times: user 1min 13s, sys: 1min 46s, total: 3min
Wall time: 49.6 s


In [18]:
test

array([[356, 376, 268, ...,   0,   0,   0],
       [457, 489, 329, ...,   0,   0,   0],
       [579, 661, 538, ...,   0,   0,   0],
       ...,
       [575, 402, 413, ...,   0,   0,   0],
       [497, 333, 358, ...,   0,   0,   0],
       [454, 293, 243, ...,   0,   0,   0]],
      shape=(428, 1198), dtype=uint16)

In [20]:
test.shape

(428, 1198)

In [19]:
import napari

viewer = napari.Viewer(title = 'testing tiling')
viewer.add_image(test)

<Image layer 'test' at 0x7fc1ef17b910>

## Downsizing images for sharing on github

In [2]:
import os
from glob import glob

import numpy as np
from skimage.transform import resize
from tifffile import imread, imwrite
from tqdm.auto import tqdm

# Set your source and output directory
src_dir = '/home/dayn/analysis/macrohet/data/untiled_images/Images/'
out_dir = '/home/dayn/analysis/macrohet/data/untiled_images/ds'
scale_factor = 1 / 5.04

os.makedirs(out_dir, exist_ok=True)

# Get all TIFF files in the directory
tiff_paths = sorted(glob(os.path.join(src_dir, "*.tiff")))

for path in tqdm(tiff_paths):
    img = imread(path)
    
    # Resize with anti-aliasing for downscaling
    downscaled = resize(
        img,
        output_shape=(int(img.shape[0] * scale_factor), int(img.shape[1] * scale_factor)),
        order=1,  # bilinear
        preserve_range=True,
        anti_aliasing=True
    ).astype(img.dtype)

    # Save to output directory with same name
    out_path = os.path.join(out_dir, os.path.basename(path))
    imwrite(out_path, downscaled)


  0%|          | 0/4050 [00:00<?, ?it/s]

In [3]:
from skimage import io

In [9]:
fns = glob('/home/dayn/analysis/macrohet/data/untiled_images/ds/*f01p01-ch1*.tif*')

In [14]:
sorted(fns)

['/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk10fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk11fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk12fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk13fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk14fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk15fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk16fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk17fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk18fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk19fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/data/untiled_images/ds/r03c05f01p01-ch1sk1fk1fl1.tiff',
 '/home/dayn/analysis/macrohet/da

In [15]:
image = np.stack([io.imread(fn) for fn in sorted(fns)])

In [16]:
import napari

In [17]:
v = napari.Viewer(title='testing downscale')
v.add_image(image)

<Image layer 'image' at 0x7fb90be15180>

### Cropping index metadata file

In [18]:
!head -n 30 ../data/untiled_images/Index.idx.xml


﻿<?xml version="1.0" encoding="utf-8"?>
<EvaluationInputData xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" Version="1" xmlns="http://www.perkinelmer.com/PEHH/HarmonyV5">
  <User>santucp</User>
  <InstrumentType>Phenix</InstrumentType>
  <Plates>
    <Plate>
      <PlateID>Live_Cell_ATB_PS_20210416</PlateID>
      <MeasurementID>6c52e048-2e42-45e6-b7af-a6d7f8f2d6cb</MeasurementID>
      <MeasurementStartTime>2021-04-16T19:09:04.1429235+01:00</MeasurementStartTime>
      <Name>Live_Cell_ATB_PS_20210416</Name>
      <PlateTypeName>96 PerkinElmer ViewPlate Glass</PlateTypeName>
      <PlateRows>8</PlateRows>
      <PlateColumns>12</PlateColumns>
      <Well id="0303" />
      <Well id="0304" />
      <Well id="0305" />
      <Well id="0306" />
      <Well id="0307" />
      <Well id="0308" />
      <Well id="0309" />
      <Well id="0310" />
      <Well id="0403" />
      <Well id="0404" />
      <Well id="0405" />
      <Well id="0406" 

In [20]:
!grep -A20 -m1 -i ".tif" ../data/untiled_images/Index.idx.xml


MH0DBstIFwoo1WuyM7m/c5Fcrw2YiutLIm1wyrRN5P3MHrViAqLym9SeGx5LgOip
nv0/ZKdKFymNc/3jBKdmUlmIx0HCo+ZnrkfIxAfxsRAIbrKFRhlvYXYGS0BaXJDv
pVDO9Gv+Qj7upBiRjwB+SzHIREJnM2PLvJvO0Wpuz/q25fD86rF333KGw+sySfiU
f4PPtLvmeDa3hXD1ZD2XT2ZDxTf9PS/yK5w/tPIr+PxUIyTJ/9KVFNUb2TUfoyRC
8WO20wN533jwhG3Z9pjW12d9kTFPAcfoO6WXGVm2L+1vUCm2C29hO8W+XssULdWT
riKORbPxPMc7lRWmgppnUCpv8I7VyztTZuZykWm5uXF6y54yzW8yKwlms2Wx4us1
fqqpD7pzZSH9v5L/r+S7x/rtSr59Vf3y0L0u3M+PntHYdUJoUW5I3vJjJ7ziKRVx
+27szPixy/PrJ14Rr6z4CUbRGF3qBu86f4TI0S/NCh8R/7H9X0NaTR9LzuvovRgp
oO/145U2lySe70eOKV7XOyo2l57ZP8bmzthslKanSO3N0p7jSNveG+liseT5pelI
+2u6R1WWP84JY5SGh4GWiHycAN6K+Ph1GldHA+Z10h3VvgMtZxXVHAsAAA==
</SkewcropParameters>
      </Entry>
      <Entry ChannelID="2">
        <SkewcropParameters>H4sIAAAAAAAEC+1WXY+iShB9N/E/EF7NLuC3ibq3UQQdFZ0BHX1roEFGaBgaUPj1
2+KM436ZG3NvNpvsG02d7upTp6qrul+OvsekKCJugHus8JlnGYTNwHKx02N1bfSp
zX7pl7pSinD8TKHAhCHyPLh631Kvsv0uOpktGMN+12foOZc1y5g91gxwDF2MIoqE
J/M/LgkiC0XIYpm4x7psX+hykLoprG9Hv5uqVyYtC1Hxn7D9GQyvLIvAy3Dgu9Aj
B

In [24]:
import os
from xml.etree import ElementTree as ET

input_path = "../data/untiled_images/Index.idx.xml"
output_path = "../data/untiled_images/Index_0305.idx.xml"
target_well_id = "0305"


tiff_dir = "../data/untiled_images/"

# Get list of actual TIFFs present
valid_tiffs = set(os.listdir(tiff_dir))

tree = ET.parse(input_path)
root = tree.getroot()

# Strip namespaces
for el in root.iter():
    if '}' in el.tag:
        el.tag = el.tag.split('}', 1)[1]

for parent in root.findall(".//*"):
    for img in list(parent.findall("Image")):  # make a static list to avoid mutation during iteration
        url_tag = img.find("URL")
        if url_tag is not None:
            tif_name = os.path.basename(url_tag.text.strip())
            if tif_name not in valid_tiffs:
                parent.remove(img)

# Write minimal file
tree.write(output_path, encoding="utf-8", xml_declaration=True)
